# Imports

In [1]:
import pandas as pd
from pathlib import Path
import os

# Data Settings

In [2]:
tfidf_path = Path(f"../words_TFIDF/")

In [3]:
import pandas as pd
import os

# Load TF-IDF for Q1 2025
tfidf_df_q12025 = pd.read_csv(f"{tfidf_path}/tfidf_q1_2025.csv")

# Load TF-IDF for Q1 2023 - Q4 2024
tfidf_df_otherq = pd.read_csv(f"{tfidf_path}/tfidf_non_q1_2025.csv")

# EPS


In [15]:
import pandas as pd
import glob
import os

# Step 1: Recursively find all EPS CSVs under data/
eps_files = glob.glob("../../data/**/EPS-*.csv", recursive=True)

# Step 2: Build company-to-DataFrame map (case-insensitive)
eps_data = {}


for filepath in eps_files:
    # Use parent folder name as company name
    company_name = os.path.basename(os.path.dirname(filepath))
    try:
        df = pd.read_csv(filepath)
        df['Quarter'] = df['Quarter'].str.strip().str.upper()
        df['Company'] = company_name
        eps_data[company_name.lower()] = df  # Store using lowercase key
    except Exception as e:
        print(f"Error loading {filepath}: {e}")

In [16]:
eps_data

{'3m':          Date    EPS Quarter Company
 0  2025-03-31   2.04  Q12025      3M
 1  2024-12-31   1.33  Q42024      3M
 2  2024-09-30   2.48  Q32024      3M
 3  2024-06-30   2.07  Q22024      3M
 4  2024-03-31   1.67  Q12024      3M
 5  2023-12-31   1.70  Q42023      3M
 6  2023-09-30  -3.74  Q32023      3M
 7  2023-06-30 -12.35  Q22023      3M
 8  2023-03-31   1.76  Q12023      3M
 9  2022-12-31   1.01  Q42022      3M,
 'amazon':          Date   EPS Quarter Company
 0  2025-03-31  1.59  Q12025  Amazon
 1  2024-12-31  1.86  Q42024  Amazon
 2  2024-09-30  1.43  Q32024  Amazon
 3  2024-06-30  1.26  Q22024  Amazon
 4  2024-03-31  0.98  Q12024  Amazon
 5  2023-12-31  1.00  Q42023  Amazon
 6  2023-09-30  0.94  Q32023  Amazon
 7  2023-06-30  0.65  Q22023  Amazon
 8  2023-03-31  0.31  Q12023  Amazon
 9  2022-12-31  0.03  Q42022  Amazon,
 'amd':          Date   EPS Quarter Company
 0  2025-03-31  0.44  Q12025     AMD
 1  2024-12-31  0.30  Q42024     AMD
 2  2024-09-30  0.47  Q32024     AMD
 3

In [17]:
# === Load TF-IDF Data ===
tfidf_train = pd.read_csv(f"{tfidf_path}/tfidf_non_q1_2025.csv")  # Q1 2023 to Q4 2024
tfidf_test = pd.read_csv(f"{tfidf_path}/tfidf_q1_2025.csv")       # Q1 2025

# === Function to get EPS label (EPS increase between two quarters) ===
def get_eps_label(company_name, q_before, q_after):
    key = company_name.lower()
    if key not in eps_data:
        return None
    df = eps_data[key ]
    try:
        eps_before = df.loc[df['quarter'] == q_before.upper(), 'EPS'].values[0]
        eps_after = df.loc[df['quarter'] == q_after.upper(), 'EPS'].values[0]
        return int(eps_after > eps_before)
    except IndexError:
        return None


# === Label training data (e.g., Q3 2024 ➜ Q4 2024 increase) ===
train_df = tfidf_train[tfidf_train["quarter"] == "Q42024"].copy()
train_df["Label"] = train_df["company"].apply(lambda c: get_eps_label(c, "Q32024", "Q42024"))
train_df = train_df.dropna(subset=["Label"])

X_train = train_df.iloc[:, 3:]  # Assuming first 3 columns are Company, Quarter, Sector
y_train = train_df["Label"].astype(int)

# === Label test data (Q4 2024 ➜ Q1 2025) ===
test_df = tfidf_test.copy()
test_df["Label"] = test_df["company"].apply(lambda c: get_eps_label(c, "Q42024", "Q12025"))
test_df = test_df.dropna(subset=["Label"])

X_test = test_df.iloc[:, 3:]
y_test = test_df["Label"].astype(int)

X_train.head()

KeyError: 'quarter'

In [8]:
print(tfidf_train.columns.tolist())

['company', 'quarter', 'Sector', 'ability', 'able', 'accelerate', 'accelerated', 'accelerating', 'acceleration', 'access', 'account', 'achieve', 'achieved', 'acquisition', 'action', 'active', 'activity', 'actual result', 'added', 'adding', 'addition', 'additionally', 'address', 'adhesive', 'adjusted', 'adjustment', 'adoption', 'advanced', 'advantage', 'advertiser', 'advertising', 'advisor', 'advisory', 'aerospace', 'affiliated', 'affiliated company', 'affo', 'ag', 'ag kgaa', 'ago', 'agreement', 'ai', 'air', 'air line', 'aircraft', 'akash', 'alastair', 'allocation', 'allow', 'allows', 'alternative', 'amazon', 'america', 'analysis', 'answer', 'answer session', 'anticipate', 'anticipated', 'app', 'application', 'approach', 'approximately', 'apps', 'architecture', 'area', 'asia', 'asked', 'asking', 'asset', 'associate', 'associated', 'assume', 'assumption', 'attractive', 'audience', 'auto', 'automation', 'automotive', 'availability', 'average', 'away', 'aws', 'backdrop', 'backlog', 'balanc